In [384]:
import pandas as pd
import numpy as np
import pymysql
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import json
user='7777'

db=pymysql.connect(
    user='root',
    passwd='111111',
    host='localhost',
    db='instagram',
    charset='utf8'
)

cursor=db.cursor(pymysql.cursors.DictCursor)
cursor2=db.cursor(pymysql.cursors.DictCursor)
cursor3=db.cursor(pymysql.cursors.DictCursor)

#like를 누른 Id조회
sql="select user.id,post_likes.post_id from user left join post_likes on user.id=post_likes.likes_id;"
cursor.execute(sql)
result=cursor.fetchall()
post_df=pd.DataFrame(result)
# sql2="select post_id,count() as count from post group by id;"
#각 게시물의 like수 조회
sql2="select post_id,count(likes_id) as count from post_likes group by post_id;"
cursor2.execute(sql2)
result2=cursor2.fetchall()
count_df=pd.DataFrame(result2)

#해당 Id가 like를 누른 post검색
sql3="select distinct(post.post_id) as post_id from post join following as f on post.id=f.following_id where f.id=%s or post.id=%s;"
cursor3.execute(sql3,(user,user))
result3=cursor3.fetchall()
follow_post_id=pd.DataFrame(result3)
follow_post_id.tail()
# post_df=pd.merge(post_df,follow_df,on="id",how="outer")
post_df=pd.merge(post_df,count_df,on="post_id",how="outer")

In [385]:
post_df['check']=post_df['post_id'].notna().astype('int').replace(0,0.5)
post_df['count']=post_df['count'].fillna(0)
post_df['post_id']=post_df['post_id'].fillna(0)
post_df['check']=post_df['check']*post_df['count']
print(post_df)

      id  post_id  count  check
0   1111      3.0    3.0    3.0
1   2222      3.0    3.0    3.0
2   3333      3.0    3.0    3.0
3   1111      5.0    2.0    2.0
4   6666      5.0    2.0    2.0
5   2222      1.0    3.0    3.0
6   3333      1.0    3.0    3.0
7   4444      1.0    3.0    3.0
8   2222      2.0    3.0    3.0
9   5555      2.0    3.0    3.0
10  6666      2.0    3.0    3.0
11  2222      6.0    2.0    2.0
12  4444      6.0    2.0    2.0
13  3333      4.0    2.0    2.0
14  5555      4.0    2.0    2.0
15  7777      0.0    0.0    0.0


In [386]:
likes=post_df[['id','post_id','check']]
likes_matrix=likes.pivot_table('check',index='id',columns='post_id')

likes_matrix.drop(likes_matrix.columns[0],axis=1,inplace=True)
print(likes_matrix)

post_id  1.0  2.0  3.0  4.0  5.0  6.0
id                                   
1111     NaN  NaN  3.0  NaN  2.0  NaN
2222     3.0  3.0  3.0  NaN  NaN  2.0
3333     3.0  NaN  3.0  2.0  NaN  NaN
4444     3.0  NaN  NaN  NaN  NaN  2.0
5555     NaN  3.0  NaN  2.0  NaN  NaN
6666     NaN  3.0  NaN  NaN  2.0  NaN
7777     NaN  NaN  NaN  NaN  NaN  NaN


In [387]:
likes_matrix=likes_matrix.fillna(0)

In [388]:
likes_matrix_T=likes_matrix.transpose()
likes_matrix_T.head()

id,1111,2222,3333,4444,5555,6666,7777
post_id,,,,,,,
1.0,0.0,3.0,3.0,3.0,0.0,0.0,0.0
2.0,0.0,3.0,0.0,0.0,3.0,3.0,0.0
3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0
5.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0


In [389]:
item_sim=cosine_similarity(likes_matrix_T,likes_matrix_T)

item_sim_df=pd.DataFrame(data=item_sim,index=likes_matrix.columns,columns=likes_matrix.columns)

item_sim_df.head(3)


post_id,1.0,2.0,3.0,4.0,5.0,6.0
post_id,,,,,,
1.0,1.000000,0.333333,0.666667,0.408248,0.000000,0.816497
2.0,0.333333,1.000000,0.333333,0.408248,0.408248,0.408248
3.0,0.666667,0.333333,1.000000,0.408248,0.408248,0.408248


In [390]:
#예측 like -> 사용자가 좋아요한 게시글에 그와 유사한 게시글의 합
def predict_likes(likes_arr,item_sim_arr):
    print(np.array([np.abs(item_sim_arr).sum(axis=1)]))
    likes_pred=likes_arr.dot(item_sim_arr)/np.array([np.abs(item_sim_arr).sum(axis=1)])
    return likes_pred

In [391]:
#결국 각각에 대한 점수들이 매겨짐 -> 유사도에 따른 예측 점수
likes_pred=predict_likes(likes_matrix.values,item_sim_df.values)
likes_pred_matrix=pd.DataFrame(data=likes_pred,index=likes_matrix.index,columns=likes_matrix.columns)
likes_pred_matrix.head(3)

[[3.22474487 2.89141154 3.22474487 2.22474487 1.81649658 2.63299316]]


post_id,1.0,2.0,3.0,4.0,5.0,6.0
id,,,,,,
1111,0.620204,0.628239,1.183503,0.550510,1.775255,0.465153
2222,2.367007,2.011646,2.113810,1.651531,1.348469,2.620204
3333,1.803708,0.974091,1.803708,2.000000,0.674235,1.395459


In [392]:
#원본데이타의 점수와 예측 점수를 비교함
#사용자가 조아요를 누른 게시글에 대해서만 예측성능평가 mse를 구함
def get_mse(pred,actual):
    pred=pred[actual.nonzero()].flatten()#0은 빼고 계산
    actual=actual[actual.nonzero()].flatten()
    print(pred)
    print(actual)
    return mean_squared_error(pred,actual)

print(get_mse(likes_pred,likes_matrix.values))

[1.18350342 1.77525513 2.36700684 2.01164604 2.11380957 2.6202041
 1.80370752 1.80370752 2.         1.43670068 1.68989795 1.31994236
 1.44948974 1.31994236 1.77525513]
[3. 2. 3. 3. 3. 2. 3. 3. 2. 3. 2. 3. 2. 3. 2.]
1.1532492742991252


In [393]:
#모든 데이터에 대해서 예측 점수 계산
def predict_likes_sim(likes_arr,item_sim_arr):
    pred=np.zeros(likes_arr.shape)
    
    for col in range(likes_arr.shape[1]):
        #유사도가 큰 순으로 모든 데이터 행렬의 index변환
        top_item=[np.argsort(item_sim_arr[:,col])[::-1]]
        #개인화된 예측 점수 계산
        for row in range(likes_arr.shape[0]):
            pred[row,col]=item_sim_arr[col,:][top_item].dot(likes_arr[row,:][top_item].T)
            pred[row,col]/=np.sum(np.abs(item_sim_arr[col,:][top_item]))
    print(pred)
    return pred

In [394]:
likes_pred=predict_likes_sim(likes_matrix.values,item_sim_df.values)
print(get_mse(likes_pred,likes_matrix.values))

likes_pred_matrix=pd.DataFrame(data=likes_pred,index=likes_matrix.index,
                              columns=likes_matrix.columns)

[[0.6202041  0.62823868 1.18350342 0.55051026 1.77525513 0.46515308]
 [2.36700684 2.01164604 2.11380957 1.65153077 1.34846923 2.6202041 ]
 [1.80370752 0.97409052 1.80370752 2.         0.67423461 1.39545923]
 [1.43670068 0.62823868 0.87340137 0.55051026 0.         1.68989795]
 [0.56329932 1.31994236 0.56329932 1.44948974 0.67423461 0.46515308]
 [0.31010205 1.31994236 0.56329932 0.55051026 1.77525513 0.46515308]
 [0.         0.         0.         0.         0.         0.        ]]
[1.18350342 1.77525513 2.36700684 2.01164604 2.11380957 2.6202041
 1.80370752 1.80370752 2.         1.43670068 1.68989795 1.31994236
 1.44948974 1.31994236 1.77525513]
[3. 2. 3. 3. 3. 2. 3. 3. 2. 3. 2. 3. 2. 3. 2.]
1.1532492742991252


<ipython-input-393-e42302de066c>:10: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row,col]=item_sim_arr[col,:][top_item].dot(likes_arr[row,:][top_item].T)
<ipython-input-393-e42302de066c>:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row,col]/=np.sum(np.abs(item_sim_arr[col,:][top_item]))


In [395]:
user_like_id=likes_matrix.loc[user,:]
user_like_id[user_like_id>0].sort_values(ascending=False)[:]

Series([], Name: 7777, dtype: float64)

In [396]:
#자신이 좋아요를 누른 것은 뺌
def get_unlike_likes(likes_matrix,userId):
    user_like=likes_matrix.loc[userId,:]
    print(user_like)
    already_like=user_like[user_like>0].index.tolist()
    
    like_list=likes_matrix.columns.tolist()
    
    unlike_list=[like for like in like_list if like not in already_like]
    
    return unlike_list

In [408]:
#좋아요를 누르지 않은 피드 중 예측률이 높은 순으로 정렬함
def recomm_like_by_userid(pred_df,userId,unlike_list):
    recomm_like=pred_df.loc[userId,unlike_list].sort_values(ascending=False)[:]
    return recomm_like

unlike_list=get_unlike_likes(likes_matrix,user)

recomm_like=recomm_like_by_userid(likes_pred_matrix,user,unlike_list)

r=pd.DataFrame(data=recomm_like.values,index=recomm_like.index,columns=['col'])
print(r)
try:
    follow_post_id_list=np.array(follow_post_id['post_id']).tolist()
except KeyError:
    follow_post_id_list=[]

recomm_like_list=np.array(recomm_like.index.values).astype('int').tolist()

follow_post_id_list=set(follow_post_id_list)
result=[x for x in recomm_like_list if x not in follow_post_id_list]
if post_df[post_df['count']==0].id.values == user:
    p=post_df[post_df['post_id'].isin(result)]
    p=p.drop_duplicates(["post_id"])
    result=np.array(p.sort_values(by=['count'],ascending=False)['post_id']).astype('int').tolist()
    result={user:result}
    result=json.dumps(result)
    print(result)
else :
    result={user:result}
    result=json.dumps(result)
    print(result)

post_id
1.0    0.0
2.0    0.0
3.0    0.0
4.0    0.0
5.0    0.0
6.0    0.0
Name: 7777, dtype: float64
         col
post_id     
6.0      0.0
5.0      0.0
4.0      0.0
3.0      0.0
2.0      0.0
1.0      0.0
{"7777": [1, 2, 5, 6, 4]}
